In [1]:
import numpy as np
import math
from SCL.src.utils import calculate_metrics, calculate_metrics_new

2025-07-28 13:35:37.708817: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-28 13:35:39.896905: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/odilon/.local/lib/python3.10/site-packages/cv2/../../lib64:
2025-07-28 13:35:39.897110: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/odilon/.local/lib/python3.10/site-packages/cv2/../../lib64

In [2]:
threshold = 0.5

In [3]:
pred_file_name = "losses/multipleModels/model_mean_part_class_mult_predictions.npy"
# pred_file_name = "losses/multipleModels/model_mean_part_class_mult_predictions.npy"
class_file_name = pred_file_name[:-15] + "cce_loss.npy"
sal_file_name = pred_file_name[:-15] + "sal_loss.npy"

In [4]:
class_file_name

'losses/multipleModels/model_mean_part_class_mult_cce_loss.npy'

In [5]:
pred = np.load(pred_file_name)
cls = np.load(class_file_name)
sal = np.load(sal_file_name)

In [6]:
print(pred.shape)
print(cls.shape)

(5, 75, 75, 20, 2)
(5, 75)


In [7]:
def round_sig(x, sig):
    return round(x, sig - int(math.floor(math.log10(abs(x)))) - 1)

In [8]:
def get_metric_dict(pred, cls, sal):
    best_metrics_dict = {"classification loss": [], "saliency loss": [], "best epoch": []}

    for idx in range(pred.shape[0]):
        epoch = np.argmin(cls[idx])
        metrics = calculate_metrics_new(pred[idx])
        for key in metrics.keys():
            new_key = key
            if new_key not in best_metrics_dict:
                best_metrics_dict[new_key] = []
            best_metrics_dict[new_key].append(metrics[key][epoch])
        best_metrics_dict["classification loss"].append(cls[idx][epoch])
        best_metrics_dict["saliency loss"].append(sal[idx][epoch])
        best_metrics_dict["best epoch"].append(epoch)
    
    mean_metrics_dict = {}
    
    for key in best_metrics_dict.keys():
        mean_metrics_dict[key] = np.mean(best_metrics_dict[key]), np.std(best_metrics_dict[key])
    
    return mean_metrics_dict

In [9]:
for key, tup in get_metric_dict(pred, cls, sal).items():
    if key in ["precision", "recall"]:
        continue
    print(key, ":", round_sig(tup[0], 3), "+-", round_sig(tup[1], 3))

classification loss : 0.107 +- 0.00236
saliency loss : 0.163 +- 0.00454
best epoch : 69.6 +- 2.94
accuracy : 0.961 +- 0.000998
f1 : 0.585 +- 0.0301
AUROC : 0.96 +- 0.00373
mAP : 0.817 +- 0.0179


In [10]:
a = np.load("losses/multipleModels/model_softmax_part_schedule_0.01_mult_predictions.npy")
del a

In [11]:
list(reversed(["0.95", "0.75", "0.5", "0.25"]))

['0.25', '0.5', '0.75', '0.95']

In [23]:
extensions = ["Mean", "Softmax", "Max Saliency", "Scheduled softmax"]
file_names = [f"losses/multipleModels/model_mean_part_class_mult_predictions.npy", "losses/multipleModels/model_softmax_part_class_ibeta_100_mult_predictions.npy", "losses/multipleModels/model_max_part_sal_mult_predictions.npy", "losses/multipleModels/model_softmax_part_schedule_0.01_mult_predictions.npy"]
# file_names = ["losses/"]
# extensions = ["softmax pooling", "mean pooling"]
# file_names = ["losses/multipleModels/model_softmax_part_schedule_0.01_mult_predictions.npy", "losses/multipleModels/model_mean_part_schedule_0.01_mult_predictions.npy"]
# file_names.append("losses/multipleModels/model_softmax_part_class_ibeta_100_mult_predictions.npy")
# extensions.append("0.0")

In [24]:
dict_li = []
for pred_file_name in file_names:
    class_file_name = pred_file_name[:-15] + "cce_loss.npy"
    sal_file_name = pred_file_name[:-15] + "sal_loss.npy"
    pred = np.load(pred_file_name)
    cls = np.load(class_file_name)
    sal = np.load(sal_file_name)
    dict_li.append(get_metric_dict(pred, cls, sal))
    

In [25]:
tables = [key for key in dict_li[0].keys() if key not in ["precision", "recall"]]

In [26]:
param_string = "Model" # "$\\beta$ LR factor"
caption_string = "different pooling layer"
label_string = "pool_"

In [27]:
for table in tables:
    print(f"\n\n{table}\n")
    print(f"\\begin{{tabular}}{{p{{1,5cm}}p{{2,5cm}}l}}\n\\toprule\n\\textbf{{{param_string}}} & \\textbf{{{table}}} & \\textbf{{STD}} \\\\ \n\\midrule")
    # print(table)
    # print("---------------------------------------------------")
    for ext, metric_dict in zip(extensions, dict_li):
        print(ext, round_sig(metric_dict[table][0], 3), round_sig(metric_dict[table][1], 3), sep=" & ", end="  \\\\ \n")
    # print("---------------------------------------------------")
    print("\\bottomrule\n\\end{tabular}")
    print(f"\\caption{{Table of the {table} metric at the epoch with the lowest classification loss for each model of {caption_string}.}}")
    print("\\label{tab:" + label_string + f"{table.replace(' ', '_')}}}")



classification loss

\begin{tabular}{p{1,5cm}p{2,5cm}l}
\toprule
\textbf{Model} & \textbf{classification loss} & \textbf{STD} \\ 
\midrule
Mean & 0.107 & 0.00236  \\ 
Softmax & 0.0983 & 0.00514  \\ 
Max Saliency & 0.142 & 0.00691  \\ 
Scheduled softmax & 0.106 & 0.00332  \\ 
\bottomrule
\end{tabular}
\caption{Table of the classification loss metric at the epoch with the lowest classification loss for each model of different pooling layer.}
\label{tab:pool_classification_loss}


saliency loss

\begin{tabular}{p{1,5cm}p{2,5cm}l}
\toprule
\textbf{Model} & \textbf{saliency loss} & \textbf{STD} \\ 
\midrule
Mean & 0.163 & 0.00454  \\ 
Softmax & 0.127 & 0.00632  \\ 
Max Saliency & 0.0566 & 0.00232  \\ 
Scheduled softmax & 0.0596 & 0.00324  \\ 
\bottomrule
\end{tabular}
\caption{Table of the saliency loss metric at the epoch with the lowest classification loss for each model of different pooling layer.}
\label{tab:pool_saliency_loss}


best epoch

\begin{tabular}{p{1,5cm}p{2,5cm}l}
\toprule